In [ ]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import sys, os, random
import importlib
import numpy as np
from sklearn.metrics import make_scorer, recall_score, fbeta_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import ComplementNB

#Class Import
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../.."))) #Allow for imports from src
from src.models import QDA_Tester
from src.models import CNB_Tester
importlib.reload(QDA_Tester) #Ensures file is uptodate!
importlib.reload(CNB_Tester) 
from src.models.QDA_Tester import QDA_Tester
from src.models.QDA_Tester import CNB_Tester

#Set Seed
os.environ["PYTHONHASHSEED"] = "1945"
random.seed(1945)
np.random.seed(1945)

In [2]:
#Complied data of convoys
#Routes examined are HX, SC, OB, ON, ONS
df = pd.read_csv('/Users/matthewplambeck/Desktop/Convoy Predictor/data/processed/Complete_Convoy_Data.csv')
df = df.drop(columns=['Unnamed: 0'])
df.shape #Test

(1174, 21)

In [3]:
#Drop unecessary/redundent features
df = df.drop(columns=['Convoy Number', 'Number of Ships Sunk', 'Depart_Date', 'Arrival/Dispersal Date', 'Number of Escorts Sunk', \
                         'Number of Stragglers Sunk', 'Total Tons of Ships Sunk', 'Escort Sink Percentage', 'Straggler Sink Percentage'])
df.reset_index(drop=True).head(3)

,Number of Ships,Number of Escort Ships,Number of Stragglers,Total Tons of Convoy,Overall Sink Percentage,Avg Number of U-Boats in Atlantic,Escort Ratio,Time At Sea (Days),Month,Year,Previous Month Avg Sink %,Approx. Sighting Range
0,5.0,2.0,0.0,22877.0,0.0,6.0,0.400000,3.0,9.0,1939.0,0.0,12.190200
1,5.0,2.0,0.0,22967.0,0.0,6.0,0.400000,5.0,9.0,1939.0,0.0,12.190200
2,7.0,4.0,0.0,21293.0,0.0,6.0,0.571429,5.0,9.0,1939.0,0.0,14.434062


In [4]:
#Convert Overall Sink Percentage to binary 1( High)
df['Risk'] = (df['Overall Sink Percentage'] > 0).astype(int) 
#Risk is binary based off whether a ship was sunk while in a convoy:  (0 = No Ships Sunk, 1 = At Least One Ship Sunk)
X = np.array(df.drop(columns=['Overall Sink Percentage', 'Risk'])) #Remove Overall Sink Percentage as it leaks data
y = df['Risk'].values #Prediction value

In [ ]:
{
    "model__alpha": [0.1, 0.5, 1.0],
    "model__norm": [True, False]
}


In [ ]:
{
    "model__alpha": [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1.0, 1.5],
    "model__norm": [True, False],
    "model__fit_prior": [True, False],
    "model__class_prior": [None,
                           [0.35, 0.65],  # bias toward minority label later if needed
                           [0.25, 0.75]]
}


In [ ]:
{
    "model__reg_param": [0.0, 0.05, 0.15],
    "model__store_covariance": [False, True]
}


In [ ]:
{
    "model__reg_param": [0.0, 0.01, 0.03, 0.05, 0.1, 0.2, 0.3],
    "model__store_covariance": [False, True],
    "model__tol": [1e-4, 1e-3, 1e-2],
    "model__priors": [None,
                      [0.92, 0.08],  # default training distribution example
                      [0.85, 0.15],  # boosted minority prior
                      [0.80, 0.20]]
}


In [ ]:

positive_label = 1  # high-risk convoy = 1
recall_scorers = {
    "recall_macro": "recall_macro",
    "recall_weighted": "recall_weighted",
    "recall_pos": make_scorer(recall_score, pos_label=positive_label),
    "f2_pos": make_scorer(fbeta_score, beta=2.0, pos_label=positive_label),
}

qda_grid = {
    "model__reg_param": [0.0, 0.01, 0.05, 0.1, 0.2, 0.3],
    "model__store_covariance": [False, True],
}
cnb_grid = {
    "model__alpha": [0.0, 0.05, 0.1, 0.25, 0.5, 1.0],
    "model__fit_prior": [True, False],
    "model__norm": [True, False],
}

qda = QDA_Tester(parameter_grid=qda_grid, cv_folds=5, random_state=1945)
qda.positive_label = positive_label
qda.auto_calibrate_threshold = True
qda.threshold_beta = 2.0  # bias threshold search toward recall
qda.train_test_split(X, y)
qda.optimize(scoring=recall_scorers, refit="recall_pos")
qda_results = qda.evaluate(show_plots=False, print_results=True)

cnb = CNB_Tester(parameter_grid=cnb_grid, cv_folds=5, random_state=1945)
cnb.positive_label = positive_label
cnb.auto_calibrate_threshold = True
cnb.train_test_split(X, y)
cnb.optimize(scoring=recall_scorers, refit="recall_pos")
cnb_results = cnb.evaluate(show_plots=False)
